In [1]:
import os
import pandas as pd

from os.path import join, expanduser

In [2]:
# Dataset files
DATASET_ROOT_DIR = 'datasets'

DATASET_DIR = 'CheXpert-v1.0-small'
IMAGES_DIR = 'train'
DATASET_ANNOTATIONS = 'train.csv'
DATASET_TRANSFORMED_ANNOTATIONS = 'Dataset_CheXpert.csv'
    
IMAGES_PATH = os.path.join(DATASET_ROOT_DIR, DATASET_DIR, IMAGES_DIR)
ANNOTATION_FILE = os.path.join(DATASET_ROOT_DIR, DATASET_DIR, DATASET_ANNOTATIONS)
TRANSFORMED_ANNOTATION_FILE = os.path.join(DATASET_ROOT_DIR, DATASET_DIR, DATASET_TRANSFORMED_ANNOTATIONS)

# Model Weights
MODELS_ROOT_DIR = 'models'
MODEL_DIR = 'captioning_model_' + DATASET_DIR
WEIGHTS_FILE = 'weights'

MODEL_DIR_PATH = os.path.join(MODELS_ROOT_DIR, MODEL_DIR)
WEIGHTS_FILE_PATH = os.path.join(MODEL_DIR_PATH, WEIGHTS_FILE)

In [3]:
annotations = pd.read_csv(ANNOTATION_FILE)
annotations

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN
223410,CheXpert-v1.0-small/train/patient64537/study1/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN
223411,CheXpert-v1.0-small/train/patient64538/study1/...,Female,0,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223412,CheXpert-v1.0-small/train/patient64539/study1/...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0


In [4]:
columns = [
        'No Finding',
        'Enlarged Cardiomediastinum', 
        'Cardiomegaly', 
        'Lung Opacity',
        'Lung Lesion',
        'Edema',
        'Consolidation',
        'Pneumonia',
        'Atelectasis',
        'Pneumothorax',
        'Pleural Effusion',
        'Pleural Other',
        'Fracture',
       ]

labels = {
    1: "Positive",
    0: "Negative",
    -1: "Uncertain",
}

def transformData(row):
    caption = ''
    
    if (row['No Finding'] == 1):
        return 'No Finding'
    
    for columnVal in columns:
        rowValue = row[columnVal]
        
        separator = ', ' if len(caption) > 0 else ''
        
        if rowValue == 0 or rowValue == 1:
            # caption += separator + labels[rowValue] + ' ' + columnVal
            caption += separator + columnVal
        else:
            caption = caption + ''
    
    return caption

In [5]:
newAnnotations = annotations.copy()

newAnnotations[['Caption']] = annotations[columns].apply(lambda row: transformData(row), axis=1)
newAnnotations = newAnnotations.drop(columns, axis = 1)
newAnnotations = newAnnotations.drop(['Support Devices'], axis = 1)
newAnnotations.head(10)

,Path,Sex,Age,Frontal/Lateral,AP/PA,Caption
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,No Finding
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,"Lung Opacity, Fracture"
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,"Lung Opacity, Fracture"
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,"Lung Opacity, Fracture"
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,"Edema, Pneumothorax"
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,No Finding
6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,No Finding
7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,No Finding
8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,No Finding
9,CheXpert-v1.0-small/train/patient00005/study2/...,Male,33,Frontal,AP,Pneumothorax


In [6]:
newAnnotations.to_csv(TRANSFORMED_ANNOTATION_FILE)